In [2]:
!nvidia-smi

Mon Aug  3 10:51:19 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN RTX           Off  | 00000000:04:00.0 Off |                  N/A |
| 40%   56C    P2   232W / 280W |  10217MiB / 24220MiB |     80%      Default |
+-------------------------------+----------------------+----------------------+
|   1  TITAN RTX           Off  | 00000000:05:00.0 Off |                  N/A |
| 67%   86C    P2   158W / 280W |  10174MiB / 24220MiB |     75%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [ ]:
######### IMPORTING NECESSARY MODULES #########
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset
import sys
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data
from torchvision import transforms, datasets, models
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm


In [ ]:
device = torch.device('cuda:4' if torch.cuda.is_available() else 'cpu')
print(device)

**Dataloading Scheme**

In [ ]:
trainlist = 'new_classification_lists/new_classification_trainlist.txt'
validlist ='new_classification_lists/new_classification_vallist.txt'
# testlist = 'new_classification_lists/new_classification_testlist.txt'

In [ ]:
# Create a mapping of Unique product Ids to Labels(0 to 31127 classes)
# output = dictionary containing mapping of each upc to a label from (0 to 31127)  

with open(trainlist, mode = 'r') as f:
    
    Y=[]
    for line in f:
        path, UPC = line[:-1].split(',')

        Y.append(UPC)
        
upc_list = sorted(set(Y))

upc_dict = { upc_list[i] :i for i in range(0, len(upc_list) ) }

In [ ]:
class mydataset():    

    def __init__(self, classification_list, upc_dict, name):

        super(mydataset).__init__()
        
        self.X = []
        self.Y = []
        
        with open(classification_list, mode = 'r') as f:
            
            for line in f:
                path, UPC = line[:-1].split(',')

                self.X.append(path)
                self.Y.append(upc_dict[UPC])
        

        if name == 'valid':
            self.transform = transforms.Compose([   
#                                                     transforms.Resize(256),
#                                                     transforms.CenterCrop(224),
                                                    transforms.RandomResizedCrop(224),
                                                    transforms.ToTensor(),
                                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                    std=[0.229, 0.224, 0.225])
                                                ])
        else:
            self.transform = transforms.Compose([   transforms.RandomResizedCrop(224),
                                                transforms.RandomHorizontalFlip(),
#                                                 transforms.RandomErasing(p=0.5)
                                                transforms.ToTensor(),
                                                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                    std=[0.229, 0.224, 0.225])
                                                                                            ])
    
    def __getitem__(self,index):
        
        image = self.X[index]        
        label = float(self.Y[index])
        
        image = (Image.open(image))
               
        image = self.transform(image)
        
        return image, torch.as_tensor(label).long()
        
  
    def __len__(self):
        return len(self.X)

In [ ]:
# #### Train Dataloader #### 
train_dataset = mydataset(trainlist, upc_dict, name='train')          
train_dataloader = data.DataLoader(train_dataset, shuffle= True, batch_size = 128, num_workers=16,pin_memory=True)


#### Validation Dataloader #### 
validation_dataset = mydataset(validlist, upc_dict, name='valid')         
validation_dataloader = data.DataLoader(validation_dataset, shuffle=False, batch_size = 128, num_workers=16,pin_memory=True)

**RESNET Architecture**

**Model Definition**

In [ ]:
# Transfer Learning
model = models.resnet50(pretrained=True)
model.fc
# for param in model.parameters():
#     param.requires_grad = False

fc_inputs = model.fc.in_features

model.fc = nn.Sequential(
    nn.Linear(fc_inputs, 31128)
#     nn.ReLU(),
#     nn.Dropout(0.4),
#     nn.Linear(256, 10),
#     nn.LogSoftmax(dim=1) # For using NLLLoss()
)


In [ ]:
model = nn.DataParallel(model,device_ids=[4,5]).to(device)
model

In [ ]:
# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4, momentum=0.9)

In [ ]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

**Training Method**

In [ ]:
def train(model, data_loader, test_loader,beta, cutmix_prob, epochs):
    model.train()

    for epoch in range(epochs):
        avg_loss = 0.0
        
        for batch_num, (feats, target) in enumerate(data_loader):
            feats, target = feats.to(device), target.to(device)
            
            
            r = np.random.rand(1)
            if beta > 0 and r < cutmix_prob:
                # generate mixed sample
                lam = np.random.beta(beta, beta)
                rand_index = torch.randperm(feats.size()[0]).to(device)
                target_a = target
                target_b = target[rand_index]
                bbx1, bby1, bbx2, bby2 = rand_bbox(feats.size(), lam)
                feats[:, :, bbx1:bbx2, bby1:bby2] = feats[rand_index, :, bbx1:bbx2, bby1:bby2]
                # adjust lambda to exactly match pixel ratio
                lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (feats.size()[-1] * feats.size()[-2]))
                # compute output
                output = model(feats)
                loss = criterion(output, target_a) * lam + criterion(output, target_b) * (1. - lam)
            else:
                # compute output
                output = model(feats)
                loss = criterion(output, target)


                                  
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            avg_loss += loss.item()

#             if batch_num % 200 == 199:
#                 print('loss', avg_loss/200)

            torch.cuda.empty_cache()
            del feats
            del target
            del loss
        
        print('Epoch: ', epoch+49)

        print('training loss = ', avg_loss/len(data_loader))
        train_loss.append(avg_loss/len(data_loader))

        ## Check performance on validation set after an Epoch
        valid_loss, valid_acc = test_classify(model, test_loader)
        print('Val Loss: {:.4f}\tVal Accuracy: {:.4f}'.format(valid_loss, valid_acc))
        v_loss.append(valid_loss)
        v_acc.append(valid_acc)

    
        
        
        #########save model checkpoint #########
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'Training_Loss_List':train_loss,
            'Validation_Loss_List':v_loss,
            'Validation_Accuracy_List': v_acc,
            'Epoch':epoch
            }, 'saved_model_checkpoints/cutmix_from_pretrained')


def test_classify(model, test_loader):
    model.eval()
    test_loss = []
    accuracy = 0
    total = 0

    for batch_num, (feats, labels) in enumerate(test_loader):
        feats, labels = feats.to(device), labels.to(device)
        outputs = model(feats)
        
        _, pred_labels = torch.max(F.softmax(outputs, dim=1), 1)
        pred_labels = pred_labels.view(-1)
        
        loss = criterion(outputs, labels.long())
        
        accuracy += torch.sum(torch.eq(pred_labels, labels)).item()
        total += len(labels)
        test_loss.extend([loss.item()]*feats.size()[0])
        del feats
        del labels

    model.train()
    return np.mean(test_loss), accuracy/total



**Hyperparameters**

In [ ]:
# Loss Function
criterion = nn.CrossEntropyLoss()

# lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 30, gamma = 0.1)


# Epochs
num_Epochs = 25

beta=1

cutmix_prob = 1

In [ ]:
train_loss= []
v_loss = []
v_acc = []

In [ ]:
train(model, train_dataloader, validation_dataloader, beta, cutmix_prob, epochs = num_Epochs)

In [ ]:
######### Load saved model from checkpoint  #########

checkpoint = torch.load('saved_model_checkpoints/cutmix_from_pretrained')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
train_loss = checkpoint['Training_Loss_List'] 
v_loss = checkpoint['Validation_Loss_List']
v_acc = checkpoint['Validation_Accuracy_List']
epoch = checkpoint['Epoch']

In [ ]:
plt.figure(figsize=(8,8))
x = np.arange(91,121)
plt.plot(x, train_loss, label = 'Training Loss')
plt.plot(x, v_loss, label = 'Validation Loss')
plt.xlabel('Epochs', fontsize =16)
plt.ylabel('Loss', fontsize =16)
plt.title('Loss v/s Epochs',fontsize =16)
plt.legend(fontsize=16)

In [ ]:
len(train_loss)

In [ ]:
v_acc

In [18]:
s='Abc'

In [19]:
s.islower()

False

In [23]:
s1='Abc'

In [24]:
s1==s

True